# test

In [3]:
import jieba
from rouge import Rouge
import numpy as np
import rouge

In [10]:
0.966 * 0.9603

0.9276498

In [4]:
help(rouge)

Help on package rouge:

NAME
    rouge

PACKAGE CONTENTS
    rouge
    rouge_score

CLASSES
    builtins.object
        rouge.rouge.FilesRouge
        rouge.rouge.Rouge
    
    class FilesRouge(builtins.object)
     |  Methods defined here:
     |  
     |  __init__(self, hyp_path, ref_path, metrics=None, stats=None, batch_lines=None)
     |      Initialize self.  See help(type(self)) for accurate signature.
     |  
     |  get_scores(self, avg=False)
     |      Calculate ROUGE scores between each pair of
     |      lines (hyp_file[i], ref_file[i]).
     |      Args:
     |        * hyp_path: hypothesis file path
     |        * ref_path: references file path
     |        * avg (False): whether to get an average scores or a list
     |  
     |  ----------------------------------------------------------------------
     |  Data descriptors defined here:
     |  
     |  __dict__
     |      dictionary for instance variables (if defined)
     |  
     |  __weakref__
     |      lis